In [3]:
from blast import read_blast_fmt10
import pandas as pd

In [9]:
model_seq_info = pd.read_parquet('../data3/interim/candidate_model_seq_info.pq')
model_seq_info['short_seq_id'] = model_seq_info['seq_id'].str[:50]
model_seq_info['seq_len'] = model_seq_info['seq'].str.len()

In [4]:
blast_results = read_blast_fmt10('../data3/interim/model_seq_blast_search.csv')

In [11]:
merged_blast_results = (blast_results
                        .merge(model_seq_info[['short_seq_id', 'seq_id', 'seq_len']]
                               .rename(columns={'short_seq_id': 'query', 
                                                'seq_len': 'qseq_len'}), 
                               how='inner', on='query')
                        .drop(columns='query')
                        .rename(columns={'seq_id': 'query'})
                        .merge(model_seq_info[['short_seq_id', 'seq_id', 'seq_len']]
                               .rename(columns={'short_seq_id': 'target', 
                                                'seq_len': 'tseq_len'}), 
                               how='inner', on='target')
                        .drop(columns='target')
                        .rename(columns={'seq_id': 'target'}))
merged_blast_results['qcov'] = merged_blast_results['alnlen']/merged_blast_results['qseq_len']
merged_blast_results['tcov'] = merged_blast_results['alnlen']/merged_blast_results['tseq_len']
merged_blast_results['fident'] = merged_blast_results['fident']/100

In [16]:
merged_blast_results.to_parquet('../data3/interim/model_seq_blast_search.pq', index=False)